<b>

==========================================================

Live Code 2

Nama  : Muhammad Iqbal Saputra

Batch : RMT-032

Objektif dari program ini adalah:
- Menampilkan nama perusahaan apa saja yang terdapat didalam data
- Mencari nilai minimum dan maksimum dari waktu pengambilan data
- Membandingkan jumlah banyaknya perjalanan antara seluruh perusahaan pada tahun 2022
- Mencari perusahaan mana yang memiliki tarif perjalanan termurah
- Mengolah data ke pandas
- Membantu programmer dalam menarik kesimpulan dan memberikan insight terhadap data yang disajikan


==========================================================

<b>

### Problem 1

In [145]:
# Proses autentikasi
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id_akun = "ace-mile-424512-u5"
client = bigquery.Client(project=project_id_akun)

Authenticated


In [20]:
# Memeriksa apakah data berhasil diambil/load dari bigQuery dengan cara menampilkan data
df = client.query('''
SELECT *
FROM `ftds-hacktiv8-project.phase0_ftds_032_rmt.taxi_fare`
''').to_dataframe()

df.head()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_km,fare,tips,payment_type,company,pickup_location,dropoff_location
0,DP-250,D-7,2021-05-18 19:30:00+00:00,2021-05-18 19:45:00+00:00,3.057746,9.00,0.0,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383)
1,DP-289,D-7,2021-05-30 15:15:00+00:00,2021-05-30 15:45:00+00:00,21.082354,35.50,0.0,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6950125892 42.001571027)
2,DP-316,D-7,2021-06-05 19:30:00+00:00,2021-06-05 19:45:00+00:00,7.242030,14.00,0.0,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.8045320063 41.9850151008)
3,DP-354,D-7,2021-06-13 13:45:00+00:00,2021-06-13 14:15:00+00:00,22.369826,37.25,0.0,Credit Card,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6559981815 41.9442266014)
4,DP-501,D-7,2021-07-07 15:15:00+00:00,2021-07-07 15:30:00+00:00,3.218680,9.00,0.0,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383)


*Menampilkan sekilas data*

In [4]:
# 1. Menampilkan semua nama perusahaan yang terdapat dalam data

df = client.query(
'''
SELECT DISTINCT company
FROM `ftds-hacktiv8-project.phase0_ftds_032_rmt.taxi_fare`
'''
).to_dataframe()

df

,company
0,Deluc Porter
1,Synctaxi Inc.
2,Atomic Inc.
3,Federway Int.
4,Rocket Cab Auto


*Dari info diatas, kita dapat mengetahui terdapat 5 perusahaan berbeda pada data yang disajikan*


In [29]:
# 2. Mencari nilai minimum dan maksimum dari waktu pengambilan data

df = client.query(
'''
SELECT
  min(TIMESTAMP_TRUNC(trip_start_timestamp,minute)) as min_time,
  max(TIMESTAMP_TRUNC(trip_start_timestamp,minute)) as max_time,
FROM `ftds-hacktiv8-project.phase0_ftds_032_rmt.taxi_fare`;
'''
).to_dataframe()

df

,min_time,max_time
0,2021-01-02 11:00:00+00:00,2022-12-31 14:15:00+00:00


In [16]:
# 3. Membandingkan jumlah banyaknya perjalanan antara seluruh perusahaan pada tahun 2022

df = client.query(
'''
SELECT company, COUNT(*) as trip_count
FROM `ftds-hacktiv8-project.phase0_ftds_032_rmt.taxi_fare`
WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2022
GROUP BY company
ORDER BY trip_count DESC
'''
).to_dataframe()

df

,company,trip_count
0,Synctaxi Inc.,1341
1,Federway Int.,1059
2,Deluc Porter,1050
3,Rocket Cab Auto,77


*Perusahaan Synctaxi Inc. memiliki transaksi terbanyak sepanjang tahun 2022*

In [71]:
# 4. Mencari perusahaan mana yang memiliki tarif perjalanan termurah

df = client.query(
'''
SELECT company,
AVG(fare/trip_km) as tarif
FROM `ftds-hacktiv8-project.phase0_ftds_032_rmt.taxi_fare`
GROUP BY company
ORDER BY tarif ASC
'''
).to_dataframe()

df

,company,tarif
0,Rocket Cab Auto,2.488383
1,Deluc Porter,3.831910
2,Atomic Inc.,3.863726
3,Synctaxi Inc.,3.945858
4,Federway Int.,4.261964


*Perusahaan Rocket Cab Auto memiliki tarif perjalanan termurah diantara 5 perusahaan*

### Problem 2


In [127]:
# Pengambilan/load data
import pandas as pd
import pandas_gbq

query = """
SELECT *
FROM `ftds-hacktiv8-project.phase0_ftds_032_rmt.taxi_fare`
"""
df = pd.read_gbq(query,project_id=project_id_akun)

# Memeriksa apakah data berhasil diambil/load dengan menampilkan data
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9286 entries, 0 to 9285
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   unique_key            9286 non-null   object             
 1   taxi_id               9286 non-null   object             
 2   trip_start_timestamp  9286 non-null   datetime64[us, UTC]
 3   trip_end_timestamp    9286 non-null   datetime64[us, UTC]
 4   trip_km               9286 non-null   float64            
 5   fare                  9286 non-null   float64            
 6   tips                  9286 non-null   float64            
 7   payment_type          9286 non-null   object             
 8   company               9286 non-null   object             
 9   pickup_location       9286 non-null   object             
 10  dropoff_location      9286 non-null   object             
dtypes: datetime64[us, UTC](2), float64(3), object(6)
memory usage: 798.1+

In [148]:
### 1. Membuat kolom baru bernama Keterangan_Waktu

df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])

## Jika hanya menggunakan if else (agar program lebih efisien)
# ============================================================

def waktu(timestamp):
  hour = timestamp.hour
  if 4 <= hour <= 9:
    return 'Pagi'
  elif 10 <= hour <= 15:
    return 'Siang'
  elif 16 <= hour <= 18:
    return 'Sore'
  elif 19 <= hour <= 23:
    return 'Malam'
  else:
    return 'Dini Hari'

## Jika menggunakan for loop
# ==========================

# df['Keterangan_Waktu'] = ''
# ket_waktu = ['Dini Hari', 'Pagi', 'Siang', 'Sore', 'Malam']
# jam = [(0,4), (4,10), (10,16), (16,19), (19,24)]

# def waktu(t):
#   if pd.notna(t):
#     time_hour = t.hour
#     for i in jam:
#       if time_hour >= i[0] and time_hour < i[1]:
#         return ket_waktu[jam.index(i)]

#   return pd.NA

df['Keterangan_Waktu'] = df['trip_start_timestamp'].apply(waktu)

df.head()




,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_km,fare,tips,payment_type,company,pickup_location,dropoff_location,Keterangan_Waktu
0,DP-250,D-7,2021-05-18 19:30:00+00:00,2021-05-18 19:45:00+00:00,3.057746,9.00,0.0,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383),Malam
1,DP-289,D-7,2021-05-30 15:15:00+00:00,2021-05-30 15:45:00+00:00,21.082354,35.50,0.0,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6950125892 42.001571027),Siang
2,DP-316,D-7,2021-06-05 19:30:00+00:00,2021-06-05 19:45:00+00:00,7.242030,14.00,0.0,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.8045320063 41.9850151008),Malam
3,DP-354,D-7,2021-06-13 13:45:00+00:00,2021-06-13 14:15:00+00:00,22.369826,37.25,0.0,Credit Card,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6559981815 41.9442266014),Siang
4,DP-501,D-7,2021-07-07 15:15:00+00:00,2021-07-07 15:30:00+00:00,3.218680,9.00,0.0,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383),Siang


*Kita berhasil membuat kolom 'Keterangan_Waktu' sekaligus mengassign value pada kolom tersebut*

In [149]:
### 2. Menghitung total jarak perjalanan yang dicapai perusahaan Federway Int. pada tahun 2022

# filter_df2021 = df[(df['company'] == 'Federway Int.') & (df['trip_start_timestamp'].dt.year == 2021)] # info tambahan
filter_df2022 = df[(df['company'] == 'Federway Int.') & (df['trip_start_timestamp'].dt.year == 2022)]

# total_jarak2021 = filter_df2021['trip_km'].sum()
total_jarak2022 = filter_df2022['trip_km'].sum()
# print(f"Total jarak yang perjalanan yang dicapai oleh perusahaan Federway Int. pada tahun 2021 adalah -> {total_jarak2021:.2f} kilometer") # info tambahan
print(f"Total jarak yang perjalanan yang dicapai oleh perusahaan Federway Int. pada tahun 2022 adalah -> {total_jarak2022:.2f} kilometer")

Total jarak yang perjalanan yang dicapai oleh perusahaan Federway Int. pada tahun 2022 adalah -> 4359.24 kilometer


### Kesimpulan akhir

- Dari data set tersebut kita dapat mengetahui ada 5 perusahaan taksi berbeda yang menyediakan layanan taksi serta tarif yang bervariasi
- Perusahaan Synctaxi Inc. memiliki trip terbanyak yang berarti perusahaan ini sangat mendominasi di masyarakat
- Walaupun perusahaan Rocket Cab Auto memiliki trip paling sedikit diantara perusahaan lain, perusahaan ini termasuk penyedia layanan termurah diantara ke-5 perusahaan tersebut
- Dengan adanya kolom baru 'Keterangan_Waktu' kita dapat menganalisis lebih lanjut, contoh. Pada perusahaan Synctaxi Inc. paling banyak memiliki trip pada malam hari yang berarti demand masyarakat pada malam hari lebih tinggi dibanding siang hari.
- Total jarak perjalanan yang dicapai oleh perusahaan Federway Int. pada tahun 2022 adalah 4359, angka ini lebih kecil dibanding angka pada tahun sebelumnya yaitu 4839. yang berarti kemungkinan ada penurunan performa dari perusahaan ini.